In [ ]:
import mercury as mr
import requests
import time
import base64

In [ ]:
#https://github.com/smurphy13IIT/Digital_Signage_Builder.git
owner = "smurphy13IIT"
repo_name = "Digital_Signage_Builder"


In [ ]:
#fork_repository(owner, repo_name, token)
def fork_repository(owner, repo_name, token):
    url = f"https://api.github.com/repos/{owner}/{repo_name}/forks"
    headers = {
        "Authorization": f"bearer {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.post(url, headers=headers)
    print(response)

def build_page(owner, repo_name, token):
    url = f"https://api.github.com/repos/{owner}/{repo_name}/pages"
    headers = {
        "Authorization": f"bearer {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    data = '''{
        "source" : {
            "branch" : "master",
            "path" : "/"
        }
    }'''
    
    response = requests.post(url, headers=headers, data=data)

    page_data = response.json()

    desc_data = {'name': 'Digital_Signage_Builder', 'homepage': page_data['html_url']}

    repo_url = f"https://api.github.com/repos/{owner}/{repo_name}"
    desc_response = requests.post(repo_url, headers=headers, json=desc_data)

    return page_data['html_url']

def write_string_to_file(owner, repo_name, file_path, token):
    url = f"https://api.github.com/repos/{owner}/{repo_name}/contents/signs/slides/{file_path}"
    headers = {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    with open(file_path, "rb") as file:
        content_base64 = base64.b64encode(file.read()).decode('utf-8')

    data = {
        "message": "Write string to file",
        "content": content_base64
    }

    try:
        response = requests.put(url, headers=headers, json=data)
        response.raise_for_status()
        print("Slide Uploaded successfully!")

    except requests.HTTPError as e:
        print(f"Error writing string to file: {e}")

def slides_builder(token, username, repo):
    headers = {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github.v3+json"
    }

    response = requests.get(f"https://api.github.com/repos/{username}/{repo}/contents/signs/slides", headers=headers)

    if response.status_code == 200:
        data = response.json()
        slide_urls = []
        for item in data:
            if item["type"] == "file":
                slide_urls.append(item["download_url"])

        slides_list = []
        slides = ''

        for i in slide_urls:
            section = f'<section data-background-image={i} data-background-size="contain, 100%"></section>'
            slides_list.append(section)

        for i in slides_list:
            slides += ("\t\t\t\t\t\t" + i + "\n")

        slides = "<div class=\"slides\">\n" + slides[:-1]

        return slides

    else:
        print("Error:", response.status_code)
        print(response.text)


def build_slides_html(slides, token, username, repo):
    headers = {
    "Authorization": f"token {token}",
    "Accept": "application/vnd.github.v3+json"
    }

    sha_response = requests.get(
        f"https://api.github.com/repos/{username}/{repo}/contents/index.html",
        headers=headers
    )

    if sha_response.status_code == 200:
        data = sha_response.json()
        if 'sha' in data:
            sha = data['sha']
        else:
            print("Blob SHA not found in response.")
    else:
        print("SHA Error:", sha_response.status_code)
        print(sha_response.text)

    response = requests.get(f"https://api.github.com/repos/{username}/{repo}/contents/signs/template.html", headers=headers)

    if response.status_code == 200:
        data = response.json()
        content = data["content"]
        # GitHub API returns the content in base64 encoding, so decode it
        content = content.encode('utf-8')
        template = base64.b64decode(content).decode('utf-8')

    content = "<div class=\"slides\">"

    slides_html = template.replace(content, slides)

    url = f"https://api.github.com/repos/{username}/{repo_name}/contents/index.html"

    content_base64 = base64.b64encode(slides_html.encode('utf-8')).decode('utf-8')

    data = {
        "message": "Write string to file",
        "content": content_base64,
        "sha": sha
    }

    try:
        response = requests.put(url, headers=headers, json=data)
        response.raise_for_status()
        print("Slideshow Updated successfully!")

    except requests.HTTPError as e:
        print(f"Error writing string to file: {e}")

In [ ]:
app = mr.App(title="Git Connection", description="App for forking the Digital Signage Builder Github repository.")
username_widget = mr.Text(value="username", label="Provide your Github Username:", rows=1)
token_widget = mr.Text(value="token", label="Provide your Github Personal Token:", rows=1)
action_select = mr.Select(value= "Update Slides", choices= ["Update Slides", "Create New Slides App"], label= "Select an action:")

In [ ]:
if action_select.value == "Create New Slides App":
    button = mr.Button(label="Create Signage Repo", style="primary")
    token = token_widget.value
    username = username_widget.value
    fork_repository(owner, repo_name, token)
    time.sleep(10)
    page_url = build_page(username, repo_name, token)
    mr.Markdown(text="""#New Signage App Successfully Created!
    Find your new slideshow at {page_url}. Don't forget to bookmark it!
    """)

elif action_select.value == "Update Slides":
    token = token_widget.value
    username = username_widget.value
    slide_file = mr.File(label="Slide Image File Upload", max_file_size="100MB")
    start_update = mr.Button(label="Begin Slides Update", style="primary")
    commit_message = "create file via PyGitHub"
    file_path = slide_file.filepath

    if start_update.clicked:
        write_string_to_file(username, repo_name, file_path, token)
        time.sleep(2)
        slides = slides_builder(token, username, repo_name)
        time.sleep(5)
        build_slides_html(slides, token, username, repo_name)
        mr.Markdown(text="""#Slides Update Complete!
        """)

# Digital Signage Builder
## IIT Galvin Library
#### Step 1: Create a Github account by <a href="https://docs.github.com/en/get-started/start-your-journey/creating-an-account-on-github">following these instructions</a>.
#### Step 2: Create a Personal Access Token <a href="https://docs.github.com/en/enterprise-server@3.9/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens">following these steps</a>.
#### NOTE: Your Github Personal Access Token is like a password. While this script will not save your token after you leave, you should treat it securely like any other password.
#### Step 3: Type your Github username in the "username" text box, then hit "enter".
#### Step 4: Paste your Github Personal Access Token in the "token" text box, then hit "enter".
#### Step 5: Click "Create Signage Repo" to copy your own version of the Digital Signage Builder installation.

NEED TO ADD:

- Method of uploading slide images for the slideshow
- Code to check existing slides folder and remove any that aren't submitted to this app